In [3]:
# Basic libraries
import numpy as np
import pandas as pd
import os

# Image processing
import cv2
from keras.preprocessing.image import ImageDataGenerator

# Model building
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth set for GPUs.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs detected.")

Memory growth set for GPUs.


In [5]:
# Path to the CSV file
data_path = "dataset/fer2013/fer2013/fer2013.csv"  # Update this path to your CSV file

# Load the dataset
df = pd.read_csv(data_path)

# Inspect the first few rows
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [6]:
# Convert pixel values from string to numpy array
def preprocess_images(df):
    images = []
    for pixel_sequence in df["pixels"]:
        img_array = np.array(pixel_sequence.split(), dtype="float32")
        img_array = img_array.reshape(48, 48, 1)  # Reshape to 48x48 and single channel
        img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
        images.append(img_array)
    return np.array(images)

# Preprocess images and extract labels
X = preprocess_images(df)
y = df["emotion"].values  # Emotion labels

# Check the shape of images and labels
print("Image data shape:", X.shape)  # Should be (num_samples, 48, 48, 1)
print("Label data shape:", y.shape)  # Should be (num_samples,)

Image data shape: (35887, 48, 48, 1)
Label data shape: (35887,)


In [7]:
# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

Training set shape: (25120, 48, 48, 1)
Validation set shape: (5383, 48, 48, 1)
Test set shape: (5384, 48, 48, 1)


In [8]:
# Define data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=15,            # Rotate images by up to 15 degrees
    width_shift_range=0.1,        # Shift images horizontally by up to 10% of the width
    height_shift_range=0.1,       # Shift images vertically by up to 10% of the height
    shear_range=0.1,              # Shear intensity (shear angle in degrees)
    zoom_range=0.1,               # Zoom in/out by up to 10%
    horizontal_flip=True,         # Randomly flip images horizontally
    fill_mode='nearest'           # Fill in new pixels with the nearest value
)

# Fit the generator to the training data
datagen.fit(X_train)

print("Data augmentation setup complete.")


Data augmentation setup complete.


In [9]:
# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(7, activation="softmax")  # 7 output classes for each emotion
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 64)        0

In [10]:
# Define callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("model/emotions.keras", save_best_only=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/30
785/785 [==============================] - 10s 5ms/step - loss: 1.7604 - accuracy: 0.2798 - val_loss: 1.5995 - val_accuracy: 0.3767
Epoch 2/30
785/785 [==============================] - 4s 5ms/step - loss: 1.5439 - accuracy: 0.4013 - val_loss: 1.3938 - val_accuracy: 0.4657
Epoch 3/30
785/785 [==============================] - 4s 5ms/step - loss: 1.4190 - accuracy: 0.4522 - val_loss: 1.2872 - val_accuracy: 0.5116
Epoch 4/30
785/785 [==============================] - 4s 5ms/step - loss: 1.3542 - accuracy: 0.4807 - val_loss: 1.2693 - val_accuracy: 0.5103
Epoch 5/30
785/785 [==============================] - 4s 5ms/step - loss: 1.3049 - accuracy: 0.5039 - val_loss: 1.2193 - val_accuracy: 0.5358
Epoch 6/30
785/785 [==============================] - 4s 5ms/step - loss: 1.2712 - accuracy: 0.5119 - val_loss: 1.1906 - val_accuracy: 0.5417
Epoch 7/30
785/785 [==============================] - 4s 5ms/step - loss: 1.2409 - accuracy: 0.5276 - val_loss: 1.1723 - val_accuracy: 0.5510
Epoch

In [11]:
# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Generate classification report
y_pred = np.argmax(model.predict(X_val), axis=1)
print(classification_report(y_val, y_pred))

169/169 [==============================] - 1s 4ms/step - loss: 1.0869 - accuracy: 0.5913
Validation Loss: 1.0869266986846924
Validation Accuracy: 0.5913059711456299
169/169 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.49      0.52      0.50       743
           1       0.78      0.26      0.39        82
           2       0.54      0.25      0.34       768
           3       0.78      0.82      0.80      1348
           4       0.45      0.56      0.50       911
           5       0.76      0.72      0.74       601
           6       0.50      0.58      0.54       930

    accuracy                           0.59      5383
   macro avg       0.61      0.53      0.54      5383
weighted avg       0.60      0.59      0.58      5383



In [13]:
try:
    model.save("model/mystery.h5")
    print("Model saved successfully.")
except Exception as e:
    print(f"Error saving model: {e}")

Model saved successfully.
